### Load preprocessed data

Run the script that downloads and processes the MovieLens data.
Uncomment it to run the download & processing script.

In [1]:
#!python ../src/download.py

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch import from_numpy
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import BatchSampler
from torch.utils.data import SequentialSampler

fh = np.load('data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
X_test = fh['test_x'].astype(np.int64)
Y_test = fh['test_y']

X_train, X_val, Y_train, Y_val = train_test_split(train_x, train_y)


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])
n_occu = int(fh['n_occu'])

# columns are user_id, item_id and other features 
# we won't use the 3rd and 4th columns
print(X_train)
print(' ')
print(Y_train)



def dataloader(*arrs, batch_size=1024):
    dataset = TensorDataset(*arrs)
    arr_size = len(arrs[0])
    bs = BatchSampler(SequentialSampler(range(arr_size)),
                      batch_size=batch_size, drop_last=False)
    return DataLoader(dataset, batch_sampler=bs, shuffle=False)
 
train = dataloader(from_numpy(X_train), from_numpy(Y_train))
test = dataloader(from_numpy(X_test), from_numpy(Y_test))
val = dataloader(from_numpy(X_val), from_numpy(Y_val))

[[3565 2826  104   18]
 [4508 2283  719   20]
 [1533 1127    7    1]
 ...
 [4480  383  466    0]
 [  45 3258  292   16]
 [6040 1251  226    6]]
 
[[2.]
 [3.]
 [4.]
 ...
 [2.]
 [1.]
 [5.]]


In [7]:
from abstract_model import AbstractModel

In [16]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl


def l2_regularize(array):
    return torch.sum(array ** 2.0)


class MF(AbstractModel):
    def __init__(self, n_user, n_item, n_occu, k=18, c_ovector=1.0,
                 c_vector=1.0, c_bias=1.0, batch_size=128):
        super().__init__()
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_bias = c_bias
        self.c_vector = c_vector
        self.c_ovector = c_ovector
        self.batch_size = batch_size
        self.save_hyperparameters()
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
        
        # **NEW: occupation vectors
        self.occu = nn.Embedding(n_occu, k)

    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # Pull out biases
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)

        # NEW: occupation-item interaction
        occu_id = inputs[:, 3]
        vector_occu = self.occu(occu_id)
        vector_user_occu = vector_user + vector_occu

        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user_occu * vector_item, dim=1)
        
        prediction = ui_interaction + biases
        return prediction

    def loss(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        log = {"mse": loss_mse}
        return loss_mse, log
    
    def reg(self):
        # Add new regularization to the biases
        reg_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        reg_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        reg_user =  l2_regularize(self.user.weight) * self.c_vector
        reg_item = l2_regularize(self.item.weight) * self.c_vector
        
        # Compute new occupation regularization
        reg_occu = l2_regularize(self.occu.weight) * self.c_ovector
        
        log = {"reg_user": reg_user, "reg_item": reg_item,
               "reg_bias_user": reg_bias_user, "reg_bias_item": reg_bias_item,
               "reg_occu": reg_occu
              }
        total = reg_user + reg_item + reg_bias_user + reg_bias_item + reg_occu
        return total, log

In [17]:
from pytorch_lightning.loggers.wandb import WandbLogger

k = 6
c_bias = 1e-3
c_vector = 1e-5
c_ovector = 1e-8
model = MF(n_user, n_item, n_occu, 
           k=k, c_bias=c_bias, c_vector=c_vector,
           c_ovector=c_ovector)

# add a logger
logger = WandbLogger(name="03_mf", project="simple_mf")

trainer = pl.Trainer(max_epochs=100, logger=logger,
                     early_stop_callback=True,
                     progress_bar_refresh_rate=1) 

GPU available: True, used: False
TPU available: False, using: 0 TPU cores


In [18]:
trainer.fit(model, train, val)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.



  | Name      | Type      | Params
----------------------------------------
0 | user      | Embedding | 36 K  
1 | item      | Embedding | 23 K  
2 | bias_user | Embedding | 6 K   
3 | bias_item | Embedding | 3 K   
4 | occu      | Embedding | 126   


/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Saving latest checkpoint..


1